In [1]:
%matplotlib notebook

from math import sqrt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pvlib

import cpvlib
from sistemas_irr_dni import genera_pot_pv, genera_pot_cpv, genera_pot_static_cpv, genera_pot_flatplate

In [2]:
A_ref = 10 # la eficiencia está ajustada para el FF con A_ref=10

lat, lon = 40.4, -3.7 # Madrid
# lat, lon = 36.857, -2.481 #Almeria
tilt = lat - 10

eff_opt_cpv = 0.83
eff_opt_pv = 0.70

spillage_factor = 0.02

diffuse_model = 'haydavies'

in_tracker = False

In [3]:
data_pvgis = pvlib.iotools.get_pvgis_tmy(lat, lon)

alt = data_pvgis[2]['location']['elevation']

data_tmy = data_pvgis[0].rename(columns={
    'Gb(n)': 'dni',
    'G(h)': 'ghi',
    'Gd(h)': 'dhi',
    'T2m': 'temp_air',
    'WS10m': 'wind_speed',
})

data = data_tmy.set_index(
    data_tmy.index.map(lambda t: t.replace(year=2010)))

location = pvlib.location.Location(
    latitude=lat, longitude=lon, altitude=alt, tz='utc')

solpos = location.get_solarposition(data.index)

# Sistema PV (plano fijo)

In [4]:
irradiance, pv_p_mp, Pdc_stc, eff_a = genera_pot_pv(A_ref, location, solpos, data, 
                                                    tilt, diffuse_model, in_singleaxis_tracker=in_tracker)

Yr = irradiance.resample('M').sum() / 1000
Ya = pv_p_mp.resample('M').sum() / Pdc_stc
Lc = Yr - Ya

PR = Ya / Yr

data_monthly = pd.concat([Yr, PR], axis='columns')
data_monthly.columns = ['Yr', 'PR']

# ax = data_monthly.plot(kind='bar', secondary_y='PR')
# ax.set_xticklabels(data.resample('M').sum().index.strftime('%b'))

print(f'Pdc_stc={Pdc_stc:.0f} eff_a={eff_a:.2%}')
print(f'PR={Ya.sum()/Yr.sum():.2}, Ya={Ya.sum():.0f} kWh/kW, Yr={Yr.sum():.0f} kWh/kW')
print(f'Total annual energy per reference area={pv_p_mp.sum()/1000:.0f} kWh/year')

Pdc_stc=2050 eff_a=20.50%
PR=0.91, Ya=1906 kWh/kW, Yr=2098 kWh/kW
Total annual energy per reference area=3908 kWh/year


# Sistema CPV (2 ejes)

In [5]:
irradiance, cpv_p_mp, Pdc_stc, eff_a = genera_pot_cpv(A_ref, location, solpos, data, tilt, eff_opt_cpv)

Yr = irradiance.resample('M').sum() / 1000
Ya = cpv_p_mp.resample('M').sum() / Pdc_stc
Lc = Yr - Ya

PR = Ya / Yr

data_monthly = pd.concat([Yr, PR], axis='columns')
data_monthly.columns = ['Yr', 'PR']

# ax = data_monthly.plot(kind='bar', secondary_y='PR')
# ax.set_xticklabels(data.resample('M').sum().index.strftime('%b'))

print(f'Pdc_stc={Pdc_stc:.0f} eff_a={eff_a:.2%}')
print(f'PR={Ya.sum()/Yr.sum():.2}, Ya={Ya.sum():.0f} kWh/kW, Yr={Yr.sum():.0f} kWh/kW')
print(f'Total annual energy per reference area={cpv_p_mp.sum()/1000:.0f} kWh/year')

Pdc_stc=3487 eff_a=34.87%
PR=0.87, Ya=1892 kWh/kW, Yr=2176 kWh/kW
Total annual energy per reference area=6597 kWh/year


C:\Users\Ruben\Documents\GitHub\StaticHybridSystem-benchmark\cpvlib.py:206: pvlibDeprecationWarning: The pvsyst_celltemp function was deprecated in pvlib 0.7 and will be removed in 0.8. Use temperature.pvsyst_cell instead. Note that the argument names for temperature.pvsyst_cell are different than for pvsyst_celltemp
  **kwargs)


# Sistema StaticCPV (plano fijo)

In [6]:
# for tilt in range(30, 40, 1):
cpv_static_irradiance, cpv_static_p_mp, Pdc_stc, eff_a, aoi = genera_pot_static_cpv(A_ref, location, solpos, data, tilt, 
                                                                                    eff_opt_cpv, in_singleaxis_tracker=in_tracker)

Yr = cpv_static_irradiance.resample('M').sum() / 1000
Ya = cpv_static_p_mp.resample('M').sum() / Pdc_stc
Lc = Yr - Ya

PR = Ya / Yr

data_monthly = pd.concat([Yr, PR], axis='columns')
data_monthly.columns = ['Yr', 'PR']

# ax = data_monthly.plot(kind='bar', secondary_y='PR')
# ax.set_xticklabels(data.resample('M').sum().index.strftime('%b'))

print(f'\nTilt={tilt:.1f}')
print(f'Pdc_stc={Pdc_stc:.0f} eff_a={eff_a:.2%}')
print(f'PR={Ya.sum()/Yr.sum():.2}, Ya={Ya.sum():.0f} kWh/kW, Yr={Yr.sum():.0f} kWh/kW')
print(f'Total annual energy per reference area={cpv_static_p_mp.sum()/1000:.0f} kWh/year')


Tilt=30.4
Pdc_stc=3487 eff_a=34.87%
PR=0.73, Ya=1092 kWh/kW, Yr=1498 kWh/kW
Total annual energy per reference area=3808 kWh/year


# Sistema FlatplatePV-Difusa(plano fijo, difusa+dni(aoi_limit)+spillage)
Solo entrada=difusa -> aoi_limit=90

In [7]:
# for tilt in range(30, 40, 1):
type_irr_input = 'diffuse'
aoi_limit = 90

cpv_irradiance_spillage = cpv_static_irradiance * spillage_factor

irradiance, pv_diffuse_p_mp, Pdc_stc, eff_a, aoi = genera_pot_flatplate(A_ref, location, solpos, data, diffuse_model, tilt, 
                                                                        aoi_limit, eff_opt_pv, cpv_irradiance_spillage, 
                                                                        type_irr_input, in_singleaxis_tracker=in_tracker)

Yr = irradiance.resample('M').sum() / 1000
Ya = pv_diffuse_p_mp.resample('M').sum() / Pdc_stc
Lc = Yr - Ya

PR = Ya / Yr

data_monthly = pd.concat([Yr, PR], axis='columns')
data_monthly.columns = ['Yr', 'PR']

# ax = data_monthly.plot(kind='bar', secondary_y='PR')
# ax.set_xticklabels(data.resample('M').sum().index.strftime('%b'))

print(f'\nTilt={tilt:.1f}')
print(f'Pdc_stc={Pdc_stc:.0f} eff_a={eff_a:.2%}')
print(f'PR={Ya.sum()/Yr.sum():.2}, Ya={Ya.sum():.0f} kWh/kW, Yr={Yr.sum():.0f} kWh/kW')
print(f'Total annual energy per reference area={pv_diffuse_p_mp.sum()/1000:.0f} kWh/year')


Tilt=30.4
Pdc_stc=1433 eff_a=14.33%
PR=0.94, Ya=566 kWh/kW, Yr=600 kWh/kW
Total annual energy per reference area=811 kWh/year


C:\Users\Ruben\Documents\GitHub\StaticHybridSystem-benchmark\cpvlib.py:713: pvlibDeprecationWarning: The pvsyst_celltemp function was deprecated in pvlib 0.7 and will be removed in 0.8. Use temperature.pvsyst_cell instead. Note that the argument names for temperature.pvsyst_cell are different than for pvsyst_celltemp
  **kwargs)


# Sistema FlatplatePV-DNI (plano fijo, difusa+dni(aoi_limit)+spillage)
Solo entrada=dni -> aoi_limit=90

In [8]:
# for tilt in range(30, 40, 1):
type_irr_input = 'dni'
aoi_limit = 90

cpv_irradiance_spillage = cpv_static_irradiance * spillage_factor

irradiance, pv_dni_p_mp, Pdc_stc, eff_a, aoi = genera_pot_flatplate(A_ref, location, solpos, data, diffuse_model, tilt, 
                                                                    aoi_limit, eff_opt_pv, cpv_irradiance_spillage, 
                                                                    type_irr_input, in_singleaxis_tracker=in_tracker)

Yr = irradiance.resample('M').sum() / 1000
Ya = pv_dni_p_mp.resample('M').sum() / Pdc_stc
Lc = Yr - Ya

PR = Ya / Yr

data_monthly = pd.concat([Yr, PR], axis='columns')
data_monthly.columns = ['Yr', 'PR']

# ax = data_monthly.plot(kind='bar', secondary_y='PR')
# ax.set_xticklabels(data.resample('M').sum().index.strftime('%b'))

print(f'\nTilt={tilt:.1f}')
print(f'Pdc_stc={Pdc_stc:.0f} eff_a={eff_a:.2%}')
print(f'PR={Ya.sum()/Yr.sum():.2}, Ya={Ya.sum():.0f} kWh/kW, Yr={Yr.sum():.0f} kWh/kW')
print(f'Total annual energy per reference area={pv_dni_p_mp.sum()/1000:.0f} kWh/year')


Tilt=30.4
Pdc_stc=1433 eff_a=14.33%
PR=2.3, Ya=1397 kWh/kW, Yr=600 kWh/kW
Total annual energy per reference area=2002 kWh/year


# Sistema Híbrido StaticCPV+Flatplate
DNI max de cada subsitema + Difusa

In [9]:
hybrid_p_mp = np.maximum(cpv_static_p_mp, pv_dni_p_mp) + pv_diffuse_p_mp

print(hybrid_p_mp.sum() / 1000)

4761.840911260151
